In [7]:
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.types import Integer, Text
from sqlalchemy.schema import ForeignKey, Column
from sqlalchemy.engine import create_engine
from sqlalchemy.orm import relationship

In [2]:
base = declarative_base()

In [3]:
base.metadata.tables

FacadeDict({})

In [33]:
sess.close()
engine.connect().close()
base.metadata.clear()
base.registry.dispose()

In [34]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker()
sess = session()
sess.bind = engine

In [35]:
class Post(base):
    __tablename__ = 'POST'
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text, default='')
    tags = relationship('PH', back_populates='post', uselist=True)

    def add(self, s, *tags):
        for tag in tags:
            t = s.query(Hashtag).filter(Hashtag.name==tag).all()
            if len(t) == 0:
                t = Hashtag(name=tag)
                s.add(t)
                s.commit()
            else:
                t = t[0]
            t.count += 1
            self.tags.append(PH(pfk=self.pk, hfk=t.pk))
        s.commit()
    # def update(self, s, *tags):
        
class Hashtag(base):
    __tablename__ = 'HASHTAG'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, default='')
    count = Column('COUNT', Integer, default=0)
    posts = relationship('PH', back_populates='tag', uselist=True)

    def plus():
        self.count += 1

    def minus():
        self.count -= 1
class PH(base):
    __tablename__ = 'PH'
    pk = Column('PK', Integer, primary_key=True)
    pfk = Column('PFK', Integer, ForeignKey('POST.PK'))
    hfk = Column('HFK', Integer, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags', uselist=False)
    tag = relationship('Hashtag', back_populates='posts', uselist=False)

In [36]:
base.metadata.create_all(engine)

2025-03-07 14:02:38,295 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:38,296 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2025-03-07 14:02:38,296 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:02:38,296 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2025-03-07 14:02:38,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:02:38,297 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2025-03-07 14:02:38,297 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:02:38,298 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2025-03-07 14:02:38,298 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:02:38,298 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PH")
2025-03-07 14:02:38,298 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:02:38,299 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PH")
2025-03-07 14:02:38,299 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 14:0

In [37]:
ph1 = PH()

In [38]:
ph1.post = Post(content='내용1')
ph1.tag = Hashtag(name='태그1')

In [39]:
sess.add(ph1)
sess.commit()

2025-03-07 14:02:39,117 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:39,118 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME", "COUNT") VALUES (?, ?)
2025-03-07 14:02:39,118 INFO sqlalchemy.engine.Engine [generated in 0.00034s] ('태그1', 0)
2025-03-07 14:02:39,119 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2025-03-07 14:02:39,119 INFO sqlalchemy.engine.Engine [generated in 0.00033s] ('내용1',)
2025-03-07 14:02:39,120 INFO sqlalchemy.engine.Engine INSERT INTO "PH" ("PFK", "HFK") VALUES (?, ?)
2025-03-07 14:02:39,120 INFO sqlalchemy.engine.Engine [generated in 0.00027s] (1, 1)
2025-03-07 14:02:39,121 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
ph1.post.content, ph1.tag.name
# INSERT Post 내용
# INSERT Hashtag 태그
# INSERT PH 내용,태그의 pk

2025-03-07 14:02:39,368 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:39,370 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE "PH"."PK" = ?
2025-03-07 14:02:39,370 INFO sqlalchemy.engine.Engine [generated in 0.00032s] (1,)
2025-03-07 14:02:39,371 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 14:02:39,371 INFO sqlalchemy.engine.Engine [generated in 0.00034s] (1,)
2025-03-07 14:02:39,372 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:39,372 INFO sqlalchemy.engine.Engine [generated in 0.00023s] (1,)


('내용1', '태그1')

In [41]:
p1 = sess.query(Post).one()

2025-03-07 14:02:39,407 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST"
2025-03-07 14:02:39,407 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ()


In [42]:
p1.content, len(p1.tags), p1.tags[0].tag.name

2025-03-07 14:02:39,438 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 14:02:39,438 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (1,)


('내용1', 1, '태그1')

In [43]:
p1.tags[0].tag.name = '새로운 태그1'

In [44]:
sess.dirty

IdentitySet([<__main__.Hashtag object at 0x10c416850>])

In [45]:
sess.commit()

2025-03-07 14:02:39,539 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "NAME"=? WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:39,540 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ('새로운 태그1', 1)
2025-03-07 14:02:39,541 INFO sqlalchemy.engine.Engine COMMIT


In [46]:
t2 = Hashtag(name='태그2')
sess.add(t2)
sess.commit()

2025-03-07 14:02:39,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:39,574 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME", "COUNT") VALUES (?, ?)
2025-03-07 14:02:39,575 INFO sqlalchemy.engine.Engine [cached since 0.4573s ago] ('태그2', 0)
2025-03-07 14:02:39,576 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
p1.tags.append(PH(pfk=p1.pk, hfk=t2.pk))

2025-03-07 14:02:39,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:39,604 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 14:02:39,604 INFO sqlalchemy.engine.Engine [generated in 0.00035s] (1,)
2025-03-07 14:02:39,605 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 14:02:39,605 INFO sqlalchemy.engine.Engine [cached since 0.1673s ago] (1,)
2025-03-07 14:02:39,606 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:39,606 INFO sqlalchemy.engine.Engine [generated in 0.00024s] (2,)


In [48]:
sess.dirty

IdentitySet([<__main__.Post object at 0x10c169d50>])

In [49]:
sess.commit()

2025-03-07 14:02:39,668 INFO sqlalchemy.engine.Engine INSERT INTO "PH" ("PFK", "HFK") VALUES (?, ?)
2025-03-07 14:02:39,669 INFO sqlalchemy.engine.Engine [cached since 0.5488s ago] (1, 2)
2025-03-07 14:02:39,670 INFO sqlalchemy.engine.Engine COMMIT


In [50]:
len(p1.tags), p1.tags[-1].tag.name

2025-03-07 14:02:39,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:39,699 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 14:02:39,699 INFO sqlalchemy.engine.Engine [cached since 0.09556s ago] (1,)
2025-03-07 14:02:39,700 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 14:02:39,701 INFO sqlalchemy.engine.Engine [cached since 0.2629s ago] (1,)
2025-03-07 14:02:39,701 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:39,701 INFO sqlalchemy.engine.Engine [cached since 0.09553s ago] (2,)


(2, '태그2')

In [51]:
new_p = Post(content='새로운 게시글')

In [52]:
sess.add(new_p)
sess.commit()

2025-03-07 14:02:40,168 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2025-03-07 14:02:40,168 INFO sqlalchemy.engine.Engine [cached since 1.05s ago] ('새로운 게시글',)
2025-03-07 14:02:40,169 INFO sqlalchemy.engine.Engine COMMIT


In [53]:
new_p.add(sess, '새로운 태그1', '새로운 태그2', '새로운 태그3')

2025-03-07 14:02:40,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:40,510 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2025-03-07 14:02:40,510 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ('새로운 태그1',)
2025-03-07 14:02:40,511 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "COUNT"=? WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:40,511 INFO sqlalchemy.engine.Engine [generated in 0.00029s] (1, 1)
2025-03-07 14:02:40,512 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 14:02:40,512 INFO sqlalchemy.engine.Engine [cached since 0.908s ago] (2,)
2025-03-07 14:02:40,513 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 14:02:40,514 INF

In [54]:
new_p.content, len(new_p.tags), new_p.tags[0].tag.name, new_p.tags[0].tag.count

2025-03-07 14:02:51,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 14:02:51,350 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2025-03-07 14:02:51,351 INFO sqlalchemy.engine.Engine [cached since 11.75s ago] (2,)
2025-03-07 14:02:51,351 INFO sqlalchemy.engine.Engine SELECT "PH"."PK" AS "PH_PK", "PH"."PFK" AS "PH_PFK", "PH"."HFK" AS "PH_HFK" 
FROM "PH" 
WHERE ? = "PH"."PFK"
2025-03-07 14:02:51,352 INFO sqlalchemy.engine.Engine [cached since 11.91s ago] (2,)
2025-03-07 14:02:51,352 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:51,352 INFO sqlalchemy.engine.Engine [cached since 11.75s ago] (1,)


('새로운 게시글', 3, '새로운 태그1', 1)

In [55]:
new_p.content, len(new_p.tags), new_p.tags[1].tag.name, new_p.tags[1].tag.count

2025-03-07 14:02:56,339 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."COUNT" AS "HASHTAG_COUNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."PK" = ?
2025-03-07 14:02:56,340 INFO sqlalchemy.engine.Engine [cached since 16.97s ago] (3,)


('새로운 게시글', 3, '새로운 태그2', 1)